In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
api_key = ('/Users/elainewei/Desktop/google-api.txt')

In [3]:
df = pd.read_csv('gpt_bio/gpt_bio_result_gpt4_new_full.csv')

In [4]:
df

,id,bio,api_out
0,C001059,a Representative from California; born in Fres...,Time Period | Location\n--- | ---\n1952-1970 |...
1,S001158,"(Brother of Kenneth Salazar), a Representative...",Time Period | Location\n--- | ---\n1953-1973 |...
2,M001155,A Representative from Florida; born in Fort My...,Time Period | Location\n--- | ---\n1967-1985 |...
3,W000797,a Representative from Florida; born in Forest ...,Time Period | Location\n--- | ---\n1966-1984 |...
4,P000591,A Representative from Georgia; born in Lansing...,Time Period | Location\n--- | ---\n1954-1972 |...
...,...,...,...
1305,B001319,"a Senator from Alabama; born in Enterprise, Al...",Time Period | Location\n--- | ---\n1982-2000 |...
1306,F000479,"a Senator from Pennsylvania; born in Reading, ...",Time Period | Location\n--- | ---\n1969-1987 |...
1307,R000618,a Senator from Nebraska; born in Nebraska City...,Time Period | Location\n--- | ---\n1964-1982 |...
1308,S001227,"a Senator from Missouri; born in Bridgeton, Mo...",Time Period | Location\n--- | ---\n1975-1993 |...


In [5]:
df['bio'] = df['bio'].apply(lambda s: s.replace('&rsquo;', "\'").replace('&amp;', '&'))

In [6]:
df_hs = df[df['bio'].str.contains('High School')]
df_hs = df_hs.reset_index(drop=True)

In [7]:
def find_highschool_sentence(row):
    bio_text = row.bio
    bio_text = bio_text.split(';')
    high_school_sentence = [s for s in bio_text if 'High School' in s][0]
    return high_school_sentence

df_hs['high_school'] = df_hs.apply(find_highschool_sentence, axis=1)
df_hs['high_school'] = df_hs['high_school'].apply(lambda s: s.replace('graduated', '').replace('from', ''))


In [8]:
def find_grad_year(text):
    match = re.search(r'[1-2][089][0-9]{2}', text)
    if match is None:
        return text, None
    else: 
        year = match.group(0)
        text = text.replace(year, '')
        year = int(year)
        return text, year

df_hs['high_school'], df_hs['grad_year'] = zip(*df_hs['high_school'].map(find_grad_year))
df_hs['ent_year'] = df_hs['grad_year'] - 4

In [9]:
df_hs

,id,bio,api_out,high_school,grad_year,ent_year
0,W000797,a Representative from Florida; born in Forest ...,Time Period | Location\n--- | ---\n1966-1984 |...,"Half Hollow Hills High School East, Dix Hil...",1984.0,1980.0
1,P000591,A Representative from Georgia; born in Lansing...,Time Period | Location\n--- | ---\n1954-1972 |...,"Dearborn High School, Dearborn, Mich.,",1972.0,1968.0
2,W000796,A Representative from Georgia; born in Atlanta...,Time Period | Location\n--- | ---\n1950-1968 |...,"Therrell High School, Atlanta, Ga.,",1968.0,1964.0
3,B001252,"A Representative from Georgia; born in Athens,...",Time Period | Location\n--- | ---\n1955-1973 |...,"Clarke Central High School, Athens, Ga.,",1973.0,1969.0
4,B001253,A Representative from Illinois; born in Chicag...,Time Period | Location\n--- | ---\n1962-1980 |...,", Maine East High School, Park Ridge, Ill.,",1980.0,1976.0
...,...,...,...,...,...,...
768,B001319,"a Senator from Alabama; born in Enterprise, Al...",Time Period | Location\n--- | ---\n1982-2000 |...,"Enterprise High School,",2000.0,1996.0
769,F000479,"a Senator from Pennsylvania; born in Reading, ...",Time Period | Location\n--- | ---\n1969-1987 |...,"Central York High School, York, Penn.,",1987.0,1983.0
770,R000618,a Senator from Nebraska; born in Nebraska City...,Time Period | Location\n--- | ---\n1964-1982 |...,"Westside High School, Omaha, Nebr.,",1982.0,1978.0
771,S001227,"a Senator from Missouri; born in Bridgeton, Mo...",Time Period | Location\n--- | ---\n1975-1993 |...,"De Smet Jesuit High School, St. Louis, Mo.,",1993.0,1989.0


In [10]:
def remove_punctuation_and_space(text):
    text = text.strip()
    if text[0] == ',':
        text = text[1:]
    if text[-1] == ',':
        text = text[:-1]
    text = text.strip()
    return text
df_hs['high_school'] = df_hs['high_school'].apply(remove_punctuation_and_space)

In [11]:
df_hs = df_hs[['id', 'bio', 'api_out', 'high_school', 'ent_year', 'grad_year']]

In [12]:
# # merge back to the original dataframe (not used)

# df_hs = df_hs.drop(columns=['bio', 'api_out'])
# df_out = df.merge(df_hs, on='id', how='left')
# df_out.to_csv()

In [13]:
df_hs.to_csv('results/high_school.csv', index=False)